In [17]:
import cv2
import time
import numpy as np
import autopy
import HandTrackingModule as htm    # 03. 導入自訂的HandTrackingModule.py

# 01. 取得鏡頭影像的基本架構
wCam, hCam = 590, 390

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

# 02. FPS時間設定
pTime = 0

# 03. 使用 HandTrackingModule.py函式擷取手部資訊
detector = htm.handDetector(detectionCon=0.75, maxHands=1)

# 06. 取得螢幕畫面大小
wScr, hScr = autopy.screen.size()

# 07. 轉換座標定位
frameR = 120 # Frame Reducution

# 11. 平滑化新增位置變數
smoothening = 8  #平滑移動值
plocX, plocY = 0, 0  #之前位置
clocX, clocY = 0, 0  #當前位置


while True:
    success, img = cap.read()
    # 水平翻轉畫面
    img = cv2.flip(img, 1) 
    
    # 03. 偵測手並傳回座標位置
    img = detector.findHands(img, draw = False)
    lmList, bbox = detector.findPosition(img)
    
    # 04. 擷取食指與中指的指尖
    if len(lmList) != 0: 
        x1, y1 = lmList[8][1:]
        x2, y2 = lmList[12][1:]
        print(x1, y1, x2, y2)
        
        # 05. 檢查那些手指向上
        fingers = detector.fingersUp()
        #print(fingers)
        
        # 07. 可控制滑鼠的範圍被限制在一個縮小畫面中
        cv2.rectangle(img, (frameR, frameR), (wCam-frameR, hCam-frameR), (255,0,255), 2)


        # 06. 移動滑鼠模式:只有食指向上     
        if fingers[1]==1 and fingers[2]==0:
            # 06. 轉換座標定位    
            # 07. 轉換座標定位    
            x3 = np.interp(x1, (frameR,wCam-frameR), (0,wScr-1))
            y3 = np.interp(y1, (frameR,hCam-frameR), (0,hScr-1))
            #print (x3, y3)
            
            # 11. 平滑化
            clocX = plocX + (x3 - plocX) / smoothening
            clocY = plocY + (y3 - plocY) / smoothening
            
            # 06. 移動滑鼠
            autopy.mouse.move(clocX, clocY)  # 11. 平滑化
            cv2.circle(img, (x1, y1), 15, (0, 255, 0), cv2.FILLED) 
        
            # 11. 平滑化
            plocX, plocY = clocX, clocY


        # 09. 點擊模式:食指與中指向上
        if fingers[1]==1 and fingers[2]==1:
            # 09. 找到兩指間的距離
            length, img, lineinfo = detector.findDistance(8, 12, img)
            cv2.putText(img, f'L:{int(length)}', (180, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 3)
            #print(length)
            # 10. 兩指距離短則點擊滑鼠
            if length < 40:
                cv2.circle(img, (lineinfo[4], lineinfo[5]), 15, (0, 0, 255), cv2.FILLED) 
                autopy.mouse.click()
        # 09. 點擊模式:食指與中指向上
        if fingers[0]==1 and fingers[1]==1:
            # 09. 找到兩指間的距離
            length, img, lineinfo = detector.findDistance(4, 8, img)
            cv2.putText(img, f'L:{int(length)}', (180, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 3)
            #print(length)
            # 10. 兩指距離短則點擊滑鼠
            if length < 40:
                cv2.circle(img, (lineinfo[4], lineinfo[5]), 15, (0, 0, 255), cv2.FILLED) 
                autopy.mouse.click(autopy.mouse.Button.RIGHT)  
       

    # 02. 顯示FPS (frame rate)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS:{int(fps)}', (wCam-120, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)


    cv2.imshow("Image", img)
    # 01. 按Q離開
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


621 94 566 142
593 98 563 239
531 136 514 307
517 148 503 312
503 157 489 316
464 153 444 295
464 150 446 288
467 151 448 288
468 149 449 286
472 156 452 288
478 160 455 289
481 160 457 291
487 168 462 301
486 165 463 302
484 167 460 300
485 171 461 303
488 196 460 326
488 200 460 326
491 203 462 327
518 217 486 340
522 219 489 341
522 221 489 340
531 230 491 344
536 233 497 346
540 232 498 349
538 232 500 348
519 225 487 343
515 224 484 340
512 223 481 342
486 222 467 343
480 221 455 345
473 221 452 344
449 222 436 344
443 220 427 345
439 221 423 343
433 218 418 341
418 209 407 334
414 207 401 335
410 207 398 334
391 201 382 327
389 200 376 328
385 200 373 327
383 200 370 327
374 198 367 326
370 196 361 324
368 194 358 321
366 188 358 316
366 187 356 314
366 186 357 313
367 178 360 306
368 177 360 304
369 178 360 304
369 178 360 303
369 178 360 304
369 178 359 305
369 178 359 304
370 177 362 303
370 177 360 304
370 182 359 304
371 182 359 301
389 176 348 247
393 175 342 214
396 178 33

293 144 346 96
292 144 345 96
286 148 346 96
285 147 344 96
293 143 345 97
293 141 344 93
295 139 344 93
295 136 344 92
300 127 315 104
290 128 306 106
285 133 349 81
289 129 348 81
292 127 350 79
283 149 364 82
271 179 367 85
271 222 357 79
263 200 378 78
264 189 377 78
264 191 376 79
265 188 377 81
265 190 375 78
265 188 377 81
268 167 373 78
272 164 369 78
284 148 364 78
299 133 346 82
305 128 324 96
285 134 324 91
286 133 345 84
288 136 350 81
287 137 352 81
287 138 352 82
284 141 354 82
284 139 353 80
284 139 355 81
285 139 355 81
287 139 355 82
286 140 354 81
281 170 304 179
376 128 444 92
367 135 424 111
354 170 388 169
361 203 408 184
369 197 412 193
377 181 459 151
380 177 463 148
377 164 466 144
381 180 477 161
383 181 475 161
379 178 467 161
364 168 452 149
337 153 414 132
348 142 432 116
349 139 434 113
379 145 461 109
376 147 464 109
380 159 470 129
369 178 469 142
367 187 471 140
369 194 474 139
365 216 484 139
361 235 489 144
357 246 493 146
366 219 485 124
363 182 473 1

In [14]:
import cv2
import mediapipe as mp
import time

class handDetector():
    def __init__(self, mode=False, maxHands = 2, model_comp = 1, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHnads = maxHands
        self.model_comp = model_comp
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        # 定義手的資訊變數
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHnads, self.model_comp, self.detectionCon, self.trackCon)
        
        # 定義畫圖單元
        self.mpDraw = mp.solutions.drawing_utils
        
        # 09. 定義手指頭的編號
        self.tipIds = [4, 8, 12, 16, 20] 

    
    def findHands(self, img, draw = False):
        # 取得multi_hand_landmarks的資料 
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        #print(results.multi_hand_landmarks)

        # 劃出關鍵點
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)  # 劃出關鍵點間的連結
        return img
     def findPosition(self, img, handNo=0, draw=True):
        # 06. 新增兩個list紀錄x與y的資訊 bbox紀錄邊界範圍與 bbox紀錄範圍
        xList = []
        yList = []
        bbox = []
        
        self.lmList = []   # 06. 將lmList修改為self.lmList

        lmList = []
        if self.results.multi_hand_landmarks:
            if (handNo == -1) :
                for hid, myHand in enumerate(self.results.multi_hand_landmarks):
                    for id, lm in enumerate(myHand.landmark):
                        #print(id, lm)  #印出檢查關鍵點座標
                        h, w, c = img.shape
                        cx, cy=int(lm.x*w), int(lm.y*h)
                        
                        # 06. 將 cx 與 cy 紀錄至 xList 與 yList
                        xList.append(cx)
                        yList.append(cy)

                        #print(id, cy, cx)  #印出轉換後的關鍵點座標
                        
                        self.lmList.append([id, cx, cy])    # 06. 將lmList修改為self.lmList

                        lmList.append([id, cx, cy])
                        if draw:
                            cv2.circle(img, (cx, cy), 5, (hid*100+100, 255-hid*100, 255), cv2.FILLED)
            else:
                if len(self.results.multi_hand_landmarks) > handNo:
                    myHand = self.results.multi_hand_landmarks[handNo]
                    for id, lm in enumerate(myHand.landmark):
                        #print(id, lm)  #印出檢查關鍵點座標
                        h, w, c = img.shape
                        cx, cy=int(lm.x*w), int(lm.y*h)
                        xList.append(cx)
                        yList.append(cy)
                        #print(id, cy, cx)  #印出轉換後的關鍵點座標
                        self.lmList.append([id, cx, cy])
                        if draw:
                            cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        # 06. 由 xlist 與 ylist 中找到最大與最小值
        if len(xList) != 0:
            xmin, xmax = min(xList), max(xList)
        else:
            xmin, xmax = 0, -20   #設定最小位置，最大值-20，因為顯示時+20
        if len(yList) != 0:
            ymin, ymax = min(yList), max(yList)
        else:
            ymin, ymax = 0, -20   #設定最小位置，最大值-20，因為顯示時+20

        # 06. 設定手的邊界範圍
        bbox = xmin, ymin, xmax, ymax
        
        #print (bbox)
        if draw:
            cv2.rectangle(img, (bbox[0]-20, bbox[1]-20), (bbox[2]+20, bbox[3]+20), (0, 255, 0), 3)

        return self.lmList, bbox
    # 09. 增加檢查手指是否向上的函式
    def fingersUp(self):
        fingers = []
        
        # Thumb
        if self.lmList[1][1] > self.lmList[0][1]:
            if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0]-1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
        else:
            if self.lmList[self.tipIds[0]][1] < self.lmList[self.tipIds[0]-1][1]:
                fingers.append(1)
            else:
                fingers.append(0)
                            
        # 4 Fingers
        for id in range(1,5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id]-2][2]:
                fingers.append(1)     
            else:
                fingers.append(0)    
        
        return fingers


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 36)

In [ ]:
import cv2
import time
import os
import HandTrackingModule as htm    # 07. 導入自訂的HandTrackingModule.py

# 01. 取得鏡頭影像的基本架構
wCam, hCam = 640, 480

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)

# 02. 設定手指圖片路徑
folderPath = "Fingerimages"
myList = os.listdir(folderPath)
print(myList)

# 03. 載入圖片至overlayList中
overlayList = []
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    print(f'{folderPath}/{imPath}')
    overlayList.append(image)
print(len(overlayList))

# 05. FPS時間設定
pTime = 0

while True:
    success, img = cap.read()
    
    # 04. 水平翻轉畫面
    img = cv2.flip(img, 1) 
    
    # 04. 載入圖片至影像中
    h, w, c = overlayList[0].shape
    img[0:h, 0:w] = overlayList[0]

    # 05. 顯示FPS (frame rate)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS:{int(fps)}', (wCam-120, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)

    
    cv2.imshow("Image", img)
    # 01. 按Q離開
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
